In [2]:
using Plots
using QuadGK
using NLopt
#using SpecialFunctions
using NLsolve
using Statistics
include("../../scaling_CGS.jl")

print_all_CGS (generic function with 2 methods)

In [410]:
#---------- Double current sheet setup ----------
base = BaseUnits("CGS")

electron_mass_scaling = 73.44610695040268 # mass ratio of 25
charge_scaling = 1e-4
eps_0_scaling = 1e3
mu_0_scaling = 1e0

scale_base_units(base, eps_0_scaling, mu_0_scaling, electron_mass_scaling, charge_scaling)


length = 1e0 #720.0847332647692
number_density= 1e9 #6.44179e10
B_flux= 7.0 #22.222
temperature   = 1e6
temperature_e = temperature
temperature_p = temperature



mu = ( (base.m_e + base.m_p)  / 2. ) /  base.m_u# assuming only protons and electrons
mass_density = number_density * 2 * mu * base.m_u
e_plasma = base.e * sqrt(4 * π * number_density / base.m_e)



#----------- These always sets the following code units --------------
#            rho=1, c=1, e_plasma=2pi
mass_density_scale = mass_density
time_scale         = 2 * pi / e_plasma
length_scale       = base.c * time_scale#
temperature_scale  = 1e0
#----------------------------------------------------------------------



#
length = length_scale

scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale, temperature_scale)
ds = 0.1
per_cell = 32

set_macro_particle_weights(scaling, ds, per_cell);
print_all_CGS(scaling)


#Figure out correct density and pressure to give c_f = 0.05
#sqrt((scaling.B_flux_real^2 + 2 * scaling.pressure_real) / scaling.mass_density_real) / scaling.c_real


 Maxwell Lorents Factors used in code:
 k_E =                                             =  1.0000e+00 
 k_B =                                             =  1.0000e+00 
 k_F =                                             =  1.0000e+00 
 k_D =                                             =  1.0000e+00 
 k_M =                                             =  1.0000e+00 
 k_H =                                             =  1.0000e+00 

 Macro particle weights:
                          ds =                     =  1.0000e-01 
                    per_cell =                     =  3.2000e+01 
               weight_scaled =                     =  7.3220e+20 
 weight_scaled_mass_electron =                     =  1.2019e-06 
   weight_scaled_mass_proton =                     =  3.0048e-05 
        weight_scaled_charge =                     =  1.0863e-05 
                 rho_n_ratio =                     =  2.3430e+25 

 Fundamental physical constants:
        Gravitational Constant [ cm^3 g^-1

In [429]:
println("mass_density scaling  = ", scaling.mass_density_scaling)
println("length scaling        = ", scaling.length_scaling)
println("time scaling          = ", scaling.time_scaling)

println("Characteristic length = ", scaling.length_real)
println("Temperature           = ", scaling.temperature_real)
println("Number density        = ", scaling.number_density_real)

println("q_e_scale            = ", charge_scaling)
println("m_e_scale            = ", electron_mass_scaling)
println("eps_0_scale          = ", eps_0_scaling)
println("mu_0_scale           = ", mu_0_scaling)

mass_density scaling  = 1.7395267739199998e-15
length scaling        = 286149.81365884363
time scaling          = 0.00030183720071927416
Characteristic length = 286149.81365884363
Temperature           = 1.0e6
Number density        = 1.0e9
q_e_scale            = 0.0001
m_e_scale            = 73.44610695040268
eps_0_scale          = 1000.0
mu_0_scale           = 1.0


In [430]:
0.0078125 / 2

0.00390625

In [403]:
0.1 / scaling.debye_len_code

13.112572131711703

In [400]:
gamma = 5. / 3.
c_s = sqrt(gamma * scaling.k_B_real * scaling.temperature_real /  scaling.m_u_real) 


c_s_ideal = sqrt(gamma * scaling.pressure_real / scaling.mass_density_real)
v_a = scaling.v_a_real

c_ms = sqrt(v_a^2 + c_s^2)

theta = 1

omega_o_k = sqrt( 0.5 * (c_ms^2 + sqrt(c_ms^4 - 4 * v_a^2 * c_s^2 * cos(theta)) ))


println( " c_s       = " , c_s / scaling.c_real)
println( " c_s_ideal = " , c_s_ideal / scaling.c_real)
println( " v_a       = " , v_a / scaling.c_real)
println( " c_ms      = ", c_ms / scaling.c_real)
println( " omega_o_k = ", omega_o_k / scaling.c_real) 

println( " beta      = ",  2 *  c_s^2 / (gamma * v_a^2))

 c_s       = 0.012417114590411072
 c_s_ideal = 0.012131923941667955
 v_a       = 0.04987879279794363
 c_ms      = 0.05140115471204507
 omega_o_k = 0.050607858083483476
 beta      = 0.0743687970065636


In [349]:
gamma = 5. / 3.
c_s = sqrt(gamma * scaling.k_B_code * scaling.temperature_code /  scaling.m_u_code) 


c_s_ideal = sqrt(gamma * scaling.pressure_code / scaling.mass_density_code)
v_a = scaling.v_a_code

c_ms = sqrt(v_a^2 + c_s^2)

theta = 0

omega_o_k = sqrt( 0.5 * (c_ms^2 + sqrt(c_ms^4 - 4 * v_a^2 * c_s^2 * cos(theta)) ))


println( " c_s       = " , c_s / scaling.c_code)
println( " c_s_ideal = " , c_s_ideal / scaling.c_code)
println( " v_a       = " , v_a / scaling.c_code)
println( " c_ms      = ", c_ms / scaling.c_code)
println( " omega_o_k = ", omega_o_k / scaling.c_code) 

println( " beta      = ",  2 *  c_s^2 / (gamma * v_a^2))

 c_s       = 0.024834229180822145
 c_s_ideal = 0.02426384788333591
 v_a       = 0.30225420531054054
 c_ms      = 0.303272721435529
 omega_o_k = 0.30225420531054054
 beta      = 0.008100986555208436


In [325]:
lambda = 32

k = 2 * pi / lambda



omega = omega_o_k * k 

0.009831745257281421

In [40]:
n = 16
ng = 3

3

In [41]:
ntot = n^3

4096

In [42]:
ntot_full = (n+ng*2)^3

10648

In [43]:
ntot_1 = (n+1 * 2)^3

5832

In [44]:
ntot_full / ntot

2.599609375

In [45]:
ntot_1 / ntot

1.423828125